In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import firm_revised

In [21]:
seed = 100

In [22]:
df = pd.read_csv(f'../data/market_{seed}.csv')
print(df)

        price      cost  product  time      Car1      Car2    mshare  \
0    3.695253  1.415698        1     1  5.543405  5.891322  0.078804   
1    3.756714  1.447384        2     1  5.278369  5.209202  0.107499   
2    3.576872  1.351142        3     1  5.424518  5.185328  0.108904   
3    3.758122  1.449088        4     1  5.844776  5.108377  0.083422   
4    3.711281  1.422723        5     1  5.004719  5.219697  0.125313   
..        ...       ...      ...   ...       ...       ...       ...   
985  3.541821  1.272978        6    99  5.121569  5.978624  0.083719   
986  3.313260  1.139542        7    99  5.670749  5.811683  0.074091   
987  3.249534  1.102120        8    99  5.825853  5.171941  0.085542   
988  3.129622  1.032779        9    99  5.136707  5.816225  0.105268   
989  3.216135  1.082681       10    99  5.575093  5.274074  0.095514   

      profits   markups  
0    0.179638  1.610199  
1    0.248250  1.595521  
2    0.242391  1.647296  
3    0.192625  1.593439  
4    

In [23]:
# All the parameters of interest 
beta = np.array([2, -0.5, -0.3])
mu = 0.5
omega = 1


# The number of firms in the market
J = 10

# The number of product characteristics 
K = 2

# The number of consumers in the market 
N = 500

# The number of period that I want to run the market for 
T = 100

In [24]:
# Unpacking the X and the price
X1 = np.array(df['Car1'][:J])
X2 = np.array(df['Car2'][:J])
all_X = np.column_stack((X1, X2))
X0 = np.ones(J)
X = np.column_stack((X0, all_X))

eq_price = np.array(df['price'])
# print(eq_price.shape)

true_share = np.array(df['mshare'])
# print(true_share)

all_est_shares = np.zeros(J*(T-1))
# print(all_est_shares)


In [25]:
# The idea is that you want to min the distance between the true share and the made up share by optimizing the fiven vector theta that determines the share 

def f(theta, true_share, N, J, X, price):
    beta = theta[:K+1]
    mu = theta[K+1]
    omega = theta[K+2]  
    # make a numpy array all estimated shares 
    for t in range(0, T-1):
        eq_price_slice = price[t*J:(t+1)*J]
        v_p = np.random.normal(0, 1, size = N)
        e = np.random.gumbel(size=N*J)
        shares_per_firm = firm_revised.share(N, J, X, v_p, eq_price_slice, beta, mu, omega, e)[0]
        all_est_shares[t*J:(t+1)*J] = shares_per_firm
    return np.linalg.norm(true_share - all_est_shares)

In [26]:
theta_0 = np.array([2., -0.5, -0.3, 0.5, 1.])
# theta_0 = np.array([1., -1., -1., 1, 1.])

In [27]:
res = minimize(f, theta_0, args=(true_share, N, J, X, eq_price), method = 'Nelder-Mead')
optim_vec = res.x
print(optim_vec)

[ 2.0075832  -0.50192565 -0.30649216  0.50207344  1.00770571]


In [28]:
all_val_b0 = []
all_val_b1 = []
all_val_b2 = []
all_val_mu = []
all_val_omega = []

for l in range(1, 10):
    
    res = minimize(f, theta_0, args=(true_share, N, J, X, eq_price), method = 'Nelder-Mead')
    optim_vec = res.x

    all_val_b0.append(optim_vec[0])
    all_val_b1.append(optim_vec[1])
    all_val_b2.append(optim_vec[2])
    all_val_mu.append(optim_vec[3])
    all_val_omega.append(optim_vec[4])

print(all_val_b0)
print(all_val_b1)
print(all_val_b2)
print(all_val_mu)
print(all_val_omega)


[2.0764837223000003, 1.9781834022004843, 2.017874629249346, 1.9793739392577319, 2.0084041989871224, 2.0154412886496265, 1.9789769790154623, 2.0464001189514596, 2.0928042053214266]
[-0.49965623057499986, -0.525033505633274, -0.5021721133411639, -0.5207057458916116, -0.4998721112701069, -0.48901203747930777, -0.49695969701979614, -0.4780000232208431, -0.4996788919039755]
[-0.303525858345, -0.29873527900847263, -0.30433678218828686, -0.2938405627686192, -0.29928469427959525, -0.3003558092151588, -0.3089227751221969, -0.3105599811932628, -0.29483505531462295]
[0.49988625132500014, 0.5012901420758519, 0.49995897578345083, 0.507965695247135, 0.5154754449967522, 0.5006941060559522, 0.51362746330992, 0.4826000470996825, 0.5088480263809028]
[1.0008153611500001, 1.0006827552669546, 0.9946545259811841, 1.013088034717415, 1.0127828030505484, 1.018155445333949, 1.0234532033157504, 1.0351998151356634, 0.9996454828632284]
